Students refer to pseudo codes of BFS, DFS, UCS, DLS, and IDS in [this link](https://drive.google.com/file/d/1q2LtrRCfemfiqyhfxNMcVJ3alvLh_pdV/view?usp=share_link) to implement the corresponding classes in TODO 1 - 5. \
Students can add supporting attributes and methods to the five classes of search strategies as needed.

# Libraries

In [601]:
import os
import heapq
from collections import deque

# Graph class

In [602]:
# Directed, weighted graphs
class Graph:
  def __init__(self):
    self.AL = dict() # adjacency list
    self.V = 0
    self.E = 0

  def __str__(self):
    res = 'V: %d, E: %d\n'%(self.V, self.E)
    for u, neighbors in self.AL.items():
      line = '%d: %s\n'%(u, str(neighbors))
      res += line
    return res

  def print(self):
    print(str(self))

  def load_from_file(self, filename):
    '''
        Example input file:
            V E
            u v w
            u v w
            u v w
            ...

        # input.txt
        7 8
        0 1 5
        0 2 6
        1 3 12
        1 4 9
        2 5 5
        3 5 8
        3 6 7
        4 6 4
    '''
    if os.path.exists(filename):
      with open(filename) as g:
        self.V, self.E = [int(it) for it in g.readline().split()]
        for line in g:
          u, v, w = [int(it) for it in line.strip().split()]
          if u not in self.AL:
            self.AL[u] = []
          self.AL[u].append((v, w))

In [603]:
g = Graph()
g.load_from_file('input.txt')
g.print()

V: 7, E: 8
0: [(1, 5), (2, 6)]
1: [(3, 2), (4, 9)]
2: [(5, 7), (7, 2)]
3: [(5, 1), (6, 7)]
4: [(6, 4), (7, 3)]
5: [(6, 2)]



# Search Strategies

In [604]:
class SearchStrategy:
  parents = dict()
  
  def search(self, g: Graph, src: int, dst: int) -> tuple:
    expanded = [] # list of expanded vertices in the traversal order
    path = [] # path from src to dst
    return expanded, path
    
  def findPath(self, src: int, dst: int) -> list:
    path = [dst]
    cur_node = dst
    
    while cur_node != src:
      cur_node = self.parents[cur_node]
      path.append(cur_node)
    path.reverse()
    
    return path

In [605]:
class BFS(SearchStrategy):
  def search(self, g: Graph, src: int, dst: int) -> tuple:
    expanded = [] # list of expanded vertices in the traversal order
    path = [] # path from src to dst
    frontier = deque([src])
    self.parents[src] = None 
    
    neighbors = []
    
    # TODO
    while frontier:
      cur_node = frontier.popleft()
      
      if cur_node == dst:
        expanded.append(cur_node)
        path = self.findPath(src, dst)
        continue
      
      expanded.append(cur_node)
      
      if cur_node in g.AL:
        neighbors = g.AL[cur_node]
      else:
        neighbors = []
      
      for (v, _) in neighbors:
        if v not in frontier and v not in expanded:
          frontier.append(v)
          self.parents[v] = cur_node
      
    return expanded, path

In [606]:
class DFS(SearchStrategy):
  def search(self, g: Graph, src: int, dst: int) -> tuple:
    expanded = [] # list of expanded vertices in the traversal order
    path = [] # path from src to dst
    frontier = deque([src])
    
    neighbors = []
    
    # TODO
    while frontier:
      cur_node = frontier.pop()
      if cur_node == dst:
        expanded.append(cur_node)
        path = self.findPath(src, dst)
        continue
      
      expanded.append(cur_node)
      
      if cur_node in g.AL:
        neighbors = g.AL[cur_node]
      else:
        neighbors = []
      
      for (v, _) in neighbors:
        if v not in frontier and v not in expanded:
          frontier.append(v)
          self.parents[v] = cur_node
      
    return expanded, path

In [607]:
class UCS(SearchStrategy):
  def search(self, g: Graph, src: int, dst: int) -> tuple:
    expanded = []  # list of expanded vertices in the traversal order
    path = []  # path from src to dst
    
    frontier = []
    path_costs = {src: 0}
    heapq.heappush(frontier, (0, src))

    neighbors = []

    while frontier:
      # print("frontier", frontier)
      weight, cur_node = heapq.heappop(frontier)
      
      if cur_node == dst:
        expanded.append(cur_node)
        path = self.findPath(src, dst)
        continue
      
      expanded.append(cur_node)
      
      if cur_node in g.AL:
        neighbors = g.AL[cur_node]
      else:
        neighbors = []
      
      for (v, w) in neighbors:
        new_cost = path_costs[cur_node] + w
        if ((v not in expanded and v not in path_costs) or
           (v in path_costs and new_cost < path_costs[v]) ):
          path_costs[v] = new_cost
          heapq.heappush(frontier, (new_cost, v))
          self.parents[v] = cur_node
        

    return expanded, path    

In [608]:
class DLS(SearchStrategy):
  def __init__(self, LIM: int):
    self.LIM = LIM

  def search(self, g: Graph, src: int, dst: int) -> tuple:
    expanded = [] # list of expanded vertices in the traversal order
    path = [] # path from src to dst
    frontier = deque([(src, 0)])
    
    neighbors = []
    
    # TODO
    while frontier:
      cur_node, cur_depth = frontier.pop()
      if cur_node == dst:
        expanded.append(cur_node)
        path = self.findPath(src, dst)
        continue
      
      expanded.append(cur_node)
      
      if cur_node in g.AL:
        neighbors = g.AL[cur_node]
      else:
        neighbors = []
      
      if cur_depth < self.LIM:
        for (v, _) in neighbors:
          if v not in frontier and v not in expanded:
            frontier.append((v, cur_depth + 1))
            self.parents[v] = cur_node
      
    return expanded, path

In [609]:
class IDS(SearchStrategy):
  def __init__(self, MAX_LIM: int):
    self.MAX_LIM = MAX_LIM

  def search(self, g: Graph, src: int, dst: int) -> tuple:
    expanded = [] # list of expanded vertices in the traversal order
    path = [] # path from src to dst
    
    # TODO 5
    for lim in range(0, self.MAX_LIM + 1):
      dls = DLS(lim)
      expanded, path = dls.search(g, src, dst)

    return expanded, path

# Evaluation

In [610]:
bfs = BFS()
dfs = DFS()
ucs = UCS()
dls = DLS(LIM=3)
ids = IDS(MAX_LIM=5)

for stg in [bfs, dfs, ucs, dls, ids]:
  print(stg)
  expanded, path = stg.search(g, 0, g.V-1)
  print(expanded)
  print(path)




[0, 1, 2, 3, 4, 5, 7, 6]
[0, 1, 3, 6]
[0, 2, 7, 5, 6, 1, 4, 3]
[0, 2, 5, 6]
[0, 1, 2, 3, 5, 7, 6, 5, 4, 6]
[0, 1, 3, 5, 6]
[0, 2, 7, 5, 6, 1, 4, 3]
[0, 2, 5, 6]
[0, 2, 7, 5, 6, 1, 4, 3]
[0, 2, 5, 6]


# Submission

*   Students download the notebook after completion
*   Rename the notebook in which inserting your student ID at the beginning. \
For example, **123456-UninformedSearch-HW.ipynb**
*   Finally, submit the file